Come costruire un sistema di raccomandazione per i libri utilizzando dati reali?
Utilizzeremo il dataset goodbooks-10k che è formato dai seguenti file:

-book_tags.csv: contiene tag (rappresentati da id) assegnati dagli utenti ai libri. Gli attributi sono: goodreads_book_id, tag_id, count.
-rating.csv: contiene i rating dei libri. Gli attributi sono: book_id, user_id, rating.
-to_read.csv: fornisce gli ID dei libri contrassegnati come “da leggere” da ciascun utente. Gli attributi sono: user_id, book_id.
-books.csv: fornisce i metadati per ogni libro. Gli attributi sono: id, book_id, best_book_id, work_id, books_count, isbn, isbn13, authors, original_publication_year, original_title.
-tags.csv: associa ID del tag a una o più parole. Gli attributi sono: tag_id, tag_name.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
import surprise

Per prima cosa importiamo i 5 dataset di interesse.

In [2]:
ratings=pd.read_csv("ratings.csv")
books=pd.read_csv("books.csv")
book_tags=pd.read_csv("book_tags.csv")
tags=pd.read_csv("tags.csv")
to_read=pd.read_csv("to_read.csv")

Dal dataset ratings ricaviamo che gli utenti che hanno lasciato almeno una recensione sono 53424, i libri recensiti sono 10.000, un utente in media ha recensito 8 libri per un totale di 980112 recensioni

In [3]:
ratings.drop_duplicates(inplace=True)

print('Abbiamo un totale',ratings.shape[0], 'rating')
print('Gli utenti che hanno eseguito almeno una recensione sono', len(ratings.user_id.unique()))
print('I libri recensiti', len(ratings.book_id.unique()), 'libri')
print("In media un utente ha recensito %d libri."%ratings.user_id.value_counts().median())
print('Il rating massimo è: %d'%ratings.rating.max(),", mentre il rating minimo è: %d"%ratings.rating.min())

ratings.head(40)

Abbiamo un totale 980112 rating
Gli utenti che hanno eseguito almeno una recensione sono 53424
I libri recensiti 10000 libri
In media un utente ha recensito 8 libri.
Il rating massimo è: 5 , mentre il rating minimo è: 1


,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4
5,1,2077,4
6,1,2487,4
7,1,2900,5
8,1,3662,4
9,1,3922,5


Il dataset books ci conferma che abbiamo 10.000 libri

In [4]:
print("In totale ci sono:",len(books), "libri")
books.head()

In totale ci sono: 10000 libri


,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [5]:
sum(books["ratings_count"])

540012351

Dal dataset book_tags ricaviamo che a un libro, in media, sono stati associati 100 tag

In [6]:
print("In media a un libro sono stati associati ",book_tags.goodreads_book_id.value_counts().median(),"tags")
book_tags.head()

In media a un libro sono stati associati  100.0 tags


,goodreads_book_id,tag_id,count
0,1,30574,167697
1,1,11305,37174
2,1,11557,34173
3,1,8717,12986
4,1,33114,12716


Il dataset tags contiene il nome dei tag associati ai libri, sono stati identificati 34252 tags possibili.

In [7]:
print("Sono stati identificati",len(tags),"possibili tags da associare a un libro")
tags.head(50)


Sono stati identificati 34252 possibili tags da associare a un libro


,tag_id,tag_name
0,0,-
1,1,--1-
2,2,--10-
3,3,--12-
4,4,--122-
5,5,--166-
6,6,--17-
7,7,--19-
8,8,--2-
9,9,--258-


Dal dataset to_read vediamo che ogni utente, in media, ha segnato 14 libri nella lista dei libri da leggere.

In [8]:
print("In media un utente ha segnato %d libri da leggere."%to_read.user_id.value_counts().median())
to_read.head()

In media un utente ha segnato 14 libri da leggere.


,user_id,book_id
0,1,112
1,1,235
2,1,533
3,1,1198
4,1,1874


In [9]:
ratings.pivot(index='user_id', columns='book_id', values='rating')


ValueError: Index contains duplicate entries, cannot reshape

In [18]:
r=ratings.iloc[:,0:2] 

a=r.drop_duplicates()
print(a)


        book_id  user_id
0             1      314
1             1      439
2             1      588
3             1     1169
4             1     1185
...         ...      ...
981751    10000    48386
981752    10000    49007
981753    10000    49383
981754    10000    50124
981755    10000    51328

[979478 rows x 2 columns]


In [26]:
b=a.merge(ratings, on=["user_id", "book_id"], how="left")




In [27]:
print(b)

        book_id  user_id  rating
0             1      314       5
1             1      439       3
2             1      588       5
3             1     1169       4
4             1     1185       4
...         ...      ...     ...
980107    10000    48386       5
980108    10000    49007       4
980109    10000    49383       5
980110    10000    50124       5
980111    10000    51328       1

[980112 rows x 3 columns]


In [12]:
d = ratings[r.duplicated(['book_id', 'user_id'])]
print(d.sort_values(by=["user_id"]))


        book_id  user_id  rating
459829     4608        7       2
883646     8946       16       3
613365     6160       25       4
883650     8946       25       3
968645     9858       42       3
...         ...      ...     ...
721911     7266    52446       4
855592     8652    52462       4
690470     6943    52767       3
655510     6587    52931       5
721809     7265    53131       3

[634 rows x 3 columns]
